# 傾向スコアを用いた逆確率重み付け方(IPTW)の実装

In [21]:
# 乱数シードを固定
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.random import randint
from scipy.special import expit
from numpy.random import randn

random.seed(1234)
np.random.seed(1234)

## データの生成(chapter4_1と同じ)

In [38]:
# データ数
num_data = 200

# 年齢
x_1 = randint(15, 75, num_data) # 15から75までの一様乱数

# 性別
x_2 = randint(0, 2, num_data) # 0か1の乱数

# ノイズの発生
e_z = randn(num_data) # 平均0、標準偏差1の正規分布

# シグモイド関数に入れる部分
z_base = x_1 + (1 - x_2) * 10 - 40 + 5*e_z

# シグモイド関数を計算(CMを見る確率)
z_prob = expit(0.1 * z_base)


# テレビCMを見たかどうかの変数(0は見ていない、1は見た)
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p = [1 - z_prob[i], z_prob[i]])[0] # CMを見る確率をweightとして0　or　1を選ぶ
    Z = np.append(Z, Z_i)
    
# ノイズの発生
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

df = pd.DataFrame({
    "年齢" : x_1,
    "性別" : x_2, 
    "CMを見た" : Z,
    "購入量" : Y
})

In [39]:
df.head()

,年齢,性別,CMを見た,購入量
0,39,1,1.0,92.528507
1,60,1,1.0,62.316669
2,28,1,0.0,83.276754
3,22,1,0.0,93.819148
4,33,0,1.0,60.642863


## 傾向スコアの実装

In [40]:
from sklearn.linear_model import LogisticRegression

In [41]:
# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数(目的変数)
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X, Z)

In [42]:
# 回帰した結果の係数を出力
print("係数beta", reg.coef_)
print("係数alpha", reg.intercept_)

係数beta [[ 0.09057178 -0.99177366]]
係数alpha [-2.38964594]


### 結果の確認 
本来の値
- β_1 = 0.1  
- β_2 = -1
- α = -3  
---
元々仮定した式  
Y^i =-x_1 + 30x_2 + 10Z + 80 + noise  
※各係数にはシグモイド関数の係数0.1　が掛け算されることに注意

## 各人の傾向スコア

In [43]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])
print("-----")
print(Z[0:5]) # 5人ほどの正解

[[0.46235142 0.53764858]
 [0.11376096 0.88623904]
 [0.69960333 0.30039667]
 [0.80040705 0.19959295]
 [0.35452151 0.64547849]]
-----
0    1.0
1    1.0
2    0.0
3    0.0
4    1.0
Name: CMを見た, dtype: float64


## ATEスコアの実装

In [44]:
ATE_i = Y / Z_pre[:, 1] * Z - Y / Z_pre[:, 0] * (1 - Z )
ATE = 1 / len(Y) * ATE_i.sum()
print("推定したATE", ATE)

推定したATE 8.23746570236478


- 元々仮定した式    
Y^i =-x_1 + 30x_2 + 10Z + 80 + noise  
--- 
テレビCMを見ると購入量が10増えるモデルを仮定していた。  
ATEは約8.2となっており、テレビCMの効果(因果の大きさ)がうまく推定されている

In [45]:
ATE

8.23746570236478